In [7]:
# Imports
from Bio import SeqIO
import json
import os

In [4]:
# Creating metadata
data = {}
for record in SeqIO.parse("linker_sequences.fasta", "fasta"):
    data[record.id] = {
        "sequence": str(record.seq)
    }

metadata = {'data': data}

In [9]:
# Saving sequences
os.makedirs('data', exist_ok=True)

# Iterating over rows
for id, d in data.items():
    seq = d['sequence']

    # Saving as FASTA format (Though one-line sequence)
    filepath = 'data/' + id + '.fasta'
    with open(filepath, 'w') as file:
        file.write('>' + id + ' ' + '\n')
        file.write(seq + '\n')

In [5]:
# Saving metadata
with open("e1a_linkers.json", 'w') as file:
    json.dump(metadata, file, indent=4)